In [5]:
import io
import os
import numpy as np
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [44]:
# Parameters

input_path = "/data/amandeep/wikidata-20211027-dwd-v3"
output_path = "/data/amandeep/wikidata-20211027-dwd-v3"

project_name = "geography-files"

files = 'label,item,monolingualtext,external_id,p279star,p31'
debug=False

In [3]:
files = files.split(',')

In [7]:
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /nas/home/amandeep
Current dir: /data/amandeep/github/kgtk-notebooks/use-cases
KGTK dir: /data/amandeep/github/kgtk-notebooks
Use-cases dir: /data/amandeep/github/kgtk-notebooks/use-cases


In [8]:
ck.print_env_variables()

kypher: kgtk query --graph-cache /data/amandeep/wikidata-20211027-dwd-v3/geography-files/temp.geography-files/wikidata.sqlite3.db
KGTK_LABEL_FILE: /data/amandeep/wikidata-20211027-dwd-v3/labels.en.tsv.gz
GRAPH: /data/amandeep/wikidata-20211027-dwd-v3
USE_CASES_DIR: /data/amandeep/github/kgtk-notebooks/use-cases
TEMP: /data/amandeep/wikidata-20211027-dwd-v3/geography-files/temp.geography-files
EXAMPLES_DIR: /data/amandeep/github/kgtk-notebooks/examples
kgtk: kgtk
STORE: /data/amandeep/wikidata-20211027-dwd-v3/geography-files/temp.geography-files/wikidata.sqlite3.db
OUT: /data/amandeep/wikidata-20211027-dwd-v3/geography-files
KGTK_OPTION_DEBUG: false
KGTK_GRAPH_CACHE: /data/amandeep/wikidata-20211027-dwd-v3/geography-files/temp.geography-files/wikidata.sqlite3.db
label: /data/amandeep/wikidata-20211027-dwd-v3/labels.en.tsv.gz
item: /data/amandeep/wikidata-20211027-dwd-v3/claims.wikibase-item.tsv.gz
monolingualtext: /data/amandeep/wikidata-20211027-dwd-v3/claims.monolingualtext.tsv.gz
ext

In [9]:
ck.load_files_into_cache()

kgtk query --graph-cache /data/amandeep/wikidata-20211027-dwd-v3/geography-files/temp.geography-files/wikidata.sqlite3.db -i "/data/amandeep/wikidata-20211027-dwd-v3/labels.en.tsv.gz" --as label  -i "/data/amandeep/wikidata-20211027-dwd-v3/claims.wikibase-item.tsv.gz" --as item  -i "/data/amandeep/wikidata-20211027-dwd-v3/claims.monolingualtext.tsv.gz" --as monolingualtext  -i "/data/amandeep/wikidata-20211027-dwd-v3/claims.external-id.tsv.gz" --as external_id  -i "/data/amandeep/wikidata-20211027-dwd-v3/derived.P279star.tsv.gz" --as p279star  -i "/data/amandeep/wikidata-20211027-dwd-v3/derived.P31.tsv.gz" --as p31  --limit 3
id	node1	label	node2	lang	rank	node2;wikidatatype
P10-label-en	P10	label	'video'@en	en		
P1000-label-en	P1000	label	'record held'@en	en		
P10000-label-en	P10000	label	'Research Vocabularies Australia ID'@en	en		


# Build Files To Reason About Geography

## `P131star`: map each node1 in `P131` to all its admin up the chain of `P131`

Removes historical admins, including historical countries.

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1000398 | P131star | Q12694 | Q1000398-P131star-Q12694 |
| Q1001008 | P131star | Q12589 | Q1001008-P131star-Q12589 |
| Q1001499 | P131star | Q214 | Q1001499-P131star-Q214 |
| Q1001995 | P131star | Q1001995 | Q1001995-P131star-Q1001995 |

Extract the graph of `P131` so that we give a smaller graph to reachable-nodes, and remove historical admins

In [10]:
kgtk("""--debug query --gc $STORE -i item
        --match '(n1)-[l:P131]->(n2)' 
        --return 'distinct n1 as node1, l.label as label, n2 as node2'
        --order-by 'n1'
        / add-id --id-style wikidata
        -o "$TEMP"/P131.tsv.gz""")

Index P131

In [11]:
kypher(""" -i "$TEMP"/P131.tsv.gz --as p131 --limit 2""")

node1	label	node2	id
P2618	P131	Q3206	P2618-P131-Q3206
P2621	P131	Q21	P2621-P131-Q21


Get all node1 in the `P131` relation, as we will use them as roots for `P131star`

In [12]:
kypher(""" -i p131
            --match 'p131: (n1)-[]->(n2)'
            --return 'distinct n1 as id' 
            -o "$TEMP"/p131.node1.tsv.gz""")

Compute `P131star`, which maps every node1 in P131 to all the admin locations that can be reached from it.

In [13]:
kgtk("""reachable-nodes
        --rootfile "$TEMP"/p131.node1.tsv.gz
        --rootfilecolumn id
        --label "P131star"
        --selflink
        -i "$TEMP"/P131.tsv.gz
        / add-id --id-style wikidata
        / sort
    -o $OUT/derived.P131star.tsv.gz""")

Index `p131star` in kypher

In [14]:
kypher(""" -i $OUT/derived.P131star.tsv.gz --as p131star --limit 2""")

node1	label	node2	id
P2618	P131star	P2618	P2618-P131star-P2618
P2618	P131star	Q3206	P2618-P131star-Q3206


Test: get all the admins of Pasadena: Q485176

In [45]:
if debug:
    !$kypher -i p131star -i label -i p31 \
    --match ' \
        p131star: (:Q485176)-[]->(admin), \
        p31: (admin)-[]->(admin_class)' \
    --opt 'labels: (admin)-[]->(admin_label)' \
    --opt 'labels: (admin_class)-[]->(admin_class_label)' \
    --return 'distinct admin as admin, admin_label as admin_label, admin_class as admin_class, admin_class_label as admin_class_label' \
    --order-by 'admin_class_label' \
    | column -t -s $'\t'

##  `derived.P131admin2` map each human settlement to its admin2 `Q13220204`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100252368 | P131admin1 | Q1588 | Q100252368-P131admin1-Q1588 |
| Q1005394 | P131admin1 | Q54171 | Q1005394-P131admin1-Q54171 |
| Q100923 | P131admin1 | Q1263 | Q100923-P131admin1-Q1263 |
| Q101111580 | P131admin1 | Q34800 | Q101111580-P131admin1-Q34800 |

Make a list of all human settlements

In [16]:
kypher(""" -i p31 -i p279star
           --match '
            p31: (human_settlement)-[]->(class),
            p279star: (class)-[]->(:Q486972)'
          --return 'distinct human_settlement as node1'
          -o "$TEMP"/human_settlement.tsv.gz""")

Index human_settlement

In [17]:
kypher(""" -i "$TEMP"/human_settlement.tsv.gz --as settlement --limit 2""")

node1
Q104214562
Q105923404


Pick out all the admins that are below admin2 `Q13220204`

In [18]:
kgtk("""--debug query --gc $STORE -i p31 -i p131star -i p279star
        --match '
        p131star: (x)-[]->(admin),
        p31: (admin)-[]->(admin_class),
        p279star: (admin_class)-[]->(:Q13220204)'
        --return 'distinct x as node1, "P131admin2" as label, admin as node2'
        / add-id --id-style wikidata
        / sort
        -o "$OUT"/derived.P131admin2.tsv.gz""")

##  `derived.P131admin1` map each human settlement to its admin1 `Q10864048`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100252368 | P131admin1 | Q1588 | Q100252368-P131admin1-Q1588 |
| Q1005394 | P131admin1 | Q54171 | Q1005394-P131admin1-Q54171 |
| Q100923 | P131admin1 | Q1263 | Q100923-P131admin1-Q1263 |
| Q101111580 | P131admin1 | Q34800 | Q101111580-P131admin1-Q34800 |

Pick out the admins that are below admin1 `Q10864048`

In [19]:
kgtk("""--debug query --gc $STORE -i p31 -i p131star -i p279star
        --match '
        p131star: (x)-[]->(admin),
        p31: (admin)-[]->(admin_class),
        p279star: (admin_class)-[]->(:Q10864048)'
        --return 'distinct x as node1, "P131admin1" as label, admin as node2'
        / add-id --id-style wikidata
        / sort
        -o "$OUT"/derived.P131admin1.tsv.gz""")

## `derived.P131country` map each settlement to its country
Removes historical country

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003172 | P131country | Q45 | Q1003172-P131country-Q45 |
| Q100701578 | P131country | Q678 | Q100701578-P131country-Q678 |
| Q1010068 | P131country | Q219 | Q1010068-P131country-Q219 |
| Q101218885 | P131country | Q822 | Q101218885-P131country-Q822 |

Pick out all the admin that are countries `Q6256`

In [20]:
kgtk("""--debug query --gc $STORE -i p31 -i p131star -i p279star
        --match '
        p131star: (x)-[]->(admin),
        p31: (admin)-[]->(admin_class),
        p279star: (admin_class)-[]->(:Q6256)'
        --return 'distinct x as node1, "P131country" as label, admin as node2'
        / add-id --id-style wikidata
        / sort
        -o "$OUT"/derived.P131country.tsv.gz""")

## `derived.P131.admin`

In [21]:
!ls "$OUT"/derived.P131*.tsv.gz

/data/amandeep/wikidata-20211027-dwd-v3/geography-files/derived.P131admin1.tsv.gz
/data/amandeep/wikidata-20211027-dwd-v3/geography-files/derived.P131admin2.tsv.gz
/data/amandeep/wikidata-20211027-dwd-v3/geography-files/derived.P131country.tsv.gz
/data/amandeep/wikidata-20211027-dwd-v3/geography-files/derived.P131star.tsv.gz


In [22]:
kgtk("""cat
        -i "$OUT"/derived.P131admin1.tsv.gz
        -i "$OUT"/derived.P131admin2.tsv.gz
        -i "$OUT"/derived.P131country.tsv.gz
        -o "$OUT"/derived.P131.admin.tsv.gz""")

Index `derived.P131.admin`

In [23]:
!$kypher -i "$OUT"/derived.P131.admin.tsv.gz --as p131admin --limit 2

node1	label	node2	id
P2618	P131admin1	Q3206	P2618-P131admin1-Q3206
P2621	P131admin1	Q21	P2621-P131admin1-Q21


Test

Something to worry about: some countries have end times:

In [102]:
!$kypher -i p31 -i labels -i qualifiers \
--match ' \
    p31: (country)-[]->(:Q6256), \
    p31: (country)-[r]->(class), \
    qualifiers: (r)-[:P582]->(endtime), \
    labels: (country)-[]->(country_label), \
    labels: (class)-[]->(class_label)' \
--return 'country, country_label, class_label, endtime' \
--limit 20

[2021-09-19 22:10:14 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node1", graph_4_c4."node2", graph_4_c5."node2", graph_7_c3."node2"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_1 AS graph_1_c2, graph_4 AS graph_4_c4, graph_4 AS graph_4_c5, graph_7 AS graph_7_c3
     ON graph_1_c1."node1" = graph_1_c2."node1"
        AND graph_1_c1."node1" = graph_4_c4."node1"
        AND graph_1_c2."id" = graph_7_c3."node1"
        AND graph_1_c2."node2" = graph_4_c5."node1"
        AND graph_1_c1."node2" = ?
        AND graph_7_c3."label" = ?
     LIMIT ?
  PARAS: ['Q6256', 'P582', 20]
---------------------------------------------
node1	node2	node2	node2
Q1000	'Gabon'@en	'French colonial empire'@en	^1960-01-01T00:00:00Z/9
Q1029	'Mozambique'@en	'overseas province of Portugal'@en	^1975-06-24T00:00:00Z/11
Q1146786	'Señorío of Cuzcatlán'@en	'country'@en	^1528-01-01T00:00:00Z/9
Q1155700	'Rattanakosin Kingdom'@en	'country'@en	^1932-06-24T00:00:00Z/11


Import the file into the kypher index and give it the alias `admin1`

In [27]:
kgtk("""--debug query --gc $STORE -i p31 -i item -i p279star
           --match '
            p31: (country)-[:P31]->(:Q6256),
            item: (settlement)-[:P17]->(country),
            p31: (settlement)-[:P31]->(a_settlement_subclass),
            p279star: (a_settlement_subclass)-[]->(:Q486972)'
           --where 'a_settlement_subclass != "Q486972"'
           --return 'settlement as node1, "P17" as label, country as node2' 
        / compact -i - --deduplicate --build-id --id-style wikidata
        -o "$TEMP"/settlement-to-country.tsv.gz""")

In [28]:
kypher("""-i "$TEMP"/settlement-to-country.tsv.gz --as settlements --limit 3""")

,node1,label,node2,id
0,Q100,P17,Q30,Q100-P17-Q30
1,Q100000,P17,Q55,Q100000-P17-Q55
2,Q1000003,P17,Q142,Q1000003-P17-Q142


In [29]:
kypher("""-i settlements -i item -i p31 -i p279star
        --match '
        settlements: (settlement)-[]->(),
        item: (settlement)-[:P131]->(admin),
        p31: (admin)-[]->(admin_class),
        p279star: (admin_class)-[]->(:Q10864048)'
        --return 'distinct settlement as node1, "P131admin1" as label, admin as node2'
        -o "$TEMP"/derived.settlement.P131admin1.direct.tsv.gz""")

In [31]:
kypher(""" -i settlements -i item -i p31 -i p279star
        --match '
        settlements: (settlement)-[]->(),
        item: (settlement)-[:P131]->(admin_a)-[:P131]->(admin),
        p31: (admin)-[]->(admin_class),
        p279star: (admin_class)-[]->(:Q10864048)'
        --return 'distinct settlement as node1, "P131admin1" as label, admin as node2'
        -o "$TEMP"/derived.settlement.P131admin1.2hop.tsv.gz""")

In [32]:
kgtk("""cat 
        -i "$TEMP"/derived.settlement.P131admin1.direct.tsv.gz
        -i "$TEMP"/derived.settlement.P131admin1.2hop.tsv.gz
        / compact -i - --deduplicate --build-id --id-style wikidata
        / sort
        -o "$OUT"/derived.settlement.P131admin1.tsv.gz""")

In [33]:
kypher(""" -i "$OUT"/derived.settlement.P131admin1.tsv.gz --as settlementadmin1 --limit 3""")

node1	label	node2	id
Q100	P131admin1	Q771	Q100-P131admin1-Q771
Q100000	P131admin1	Q1093	Q100000-P131admin1-Q1093
Q1000003	P131admin1	Q16987	Q1000003-P131admin1-Q16987


# Aliases for cities

### `derived.alias.settlement.admin1.full.tsv`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100349105 | alias | 'Colonnacce Roman villa, Lazio'@en | Q100349105-alias-44bd19 |
| Q1007434 | alias | 'Frangovo, Struga Municipality'@en | Q1007434-alias-1a7cbb |
| Q101071309 | alias | '50-76 Stryiska Street, Lviv, Lwów Voivodeship'@en | Q101071309-alias-6c36ec |
| Q1012440 | alias | 'Ranong, Ranong'@en | Q1012440-alias-2af390 |

In [34]:
kgtk("""--debug query --gc $STORE -i settlementadmin1 -i label
        --match '
        settlementadmin1: (settlement)-[]->(admin1),
        label: (settlement)-[]->(settlement_label),
        label: (admin1)-[]->(admin1_label)'
        --return 'distinct settlement as node1, "alias" as label,
        kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), kgtk_lqstring_text(admin1_label))) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`,
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort
        -o "$OUT"/derived.alias.settlement.admin1.full.tsv.gz""")

### `derived.alias.settlement.admin1.short.tsv`
Use the  short name (`P1813`) of the admin as the label

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1012017 | alias | 'Plainfield, IL'@en | Q1012017-alias-28be4a |
| Q1020218 | alias | 'Princeton, C.-B.'@en | Q1020218-alias-f54afb |
| Q1029922 | alias | 'Camp Pendleton North, Cal.'@en | Q1029922-alias-3876ac |
| Q1052502 | alias | 'Cedar Grove, IN'@en | Q1052502-alias-b1d922 |

In [36]:
kgtk("""--debug query --gc $STORE -i settlementadmin1 -i monolingualtext -i label
        --match '
        settlementadmin1: (settlement)-[]->(admin1),
        label: (settlement)-[]->(settlement_label),
        monolingualtext: (admin1)-[:P1813]->(admin1_short_label)'
        --return 'distinct settlement as node1, "alias" as label,
        kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), kgtk_lqstring_text(admin1_short_label))) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`,
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort
        -o "$OUT"/derived.alias.settlement.admin1.short.tsv.gz""")

### `derived.alias.settlement.admin1.iso.tsv`

Use the iso code (`P300`) of the admin as the label, after removing the country part of the code

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1003425 | alias | 'Fourneaux, P'@en | Q1003425-alias-f75f90 |
| Q1007427 | alias | 'La Lande-de-Fronsac, B'@en | Q1007427-alias-0eaf01 |
| Q101073 | alias | 'Foresto Sparso, 25'@en | Q101073-alias-b6bf2f |
| Q1012377 | alias | 'Fabryczna, LB'@en | Q1012377-alias-e30d31 |

In [37]:
kgtk("""--debug query --gc $STORE -i settlementadmin1 -i external_id -i label
        --match '
        settlementadmin1: (settlement)-[]->(admin1),
        label: (settlement)-[]->(settlement_label),
        external_id: (admin1)-[:P300]->(iso_code)'
        --return 'distinct settlement as node1, "alias" as label,
        kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(settlement_label), substr(kgtk_unstringify(iso_code), 4))) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`,
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort 
        -o "$OUT"/derived.alias.settlement.admin1.iso.tsv.gz""")

### `derived.alias.city.country.tsv`

Produce "city, country" aliases for big cities

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q1023481 | alias | 'Ocilla, United States of America'@en | Q1023481-alias-51eee8 |
| Q1160417 | alias | 'Lébény, Hungary'@en | Q1160417-alias-e1447f |
| Q1341 | alias | 'Tolyatti, Soviet Union'@en | Q1341-alias-e387a3 |
| Q151920 | alias | 'Tiberias, Mandatory Palestine'@en | Q151920-alias-0fb665 |

Produce city/country aliases for all cities (Q515) and all countries

In [38]:
# Q515: city
kgtk("""--debug query --gc $STORE -i p31 -i p279star -i item -i label
        --match ' 
        p31: (city)-[]->(city_class),
        p279star: (city_class)-[]->(:Q515),
        item: (city)-[:P17]->(country),
        label: (city)-[]->(city_label),
        label: (country)-[]->(country_label)'
        --return 'distinct city as node1, "alias" as label,
        kgtk_stringify(printf("%s, %s", kgtk_lqstring_text(city_label), kgtk_lqstring_text(country_label), 4)) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`,
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort 
        -o "$OUT"/derived.alias.city.country.tsv.gz""")

### `derived.alias.city.us.tsv`

For the US, produce special files that use `USA` and `United States` in addition to the official name `United States of America`

| node1 | label | node2 | id |
| -- | -- | -- | -- |
| Q100 | alias | 'Boston, United States'@en | Q100-alias-d454d3 |
| Q100 | alias | 'Boston, USA'@en | Q100-alias-fd19d4 |
| Q1000030 | alias | 'Orange City, USA'@en | Q1000030-alias-63cb74 |
| Q1000030 | alias | 'Orange City, United States'@en | Q1000030-alias-a00c53 |
| Q1000065 | alias | 'Neosho, USA'@en | Q1000065-alias-3631c0 |
| Q1000065 | alias | 'Neosho, United States'@en | Q1000065-alias-6be460 |

In [39]:
kgtk("""--debug query --gc $STORE -i p31 -i p279star -i item -i label
        --match '
        p31: (city)-[]->(city_class),
        p279star: (city_class)-[]->(:Q515),
        item: (city)-[:P17]->(:Q30),
        label: (city)-[]->(city_label)'
        --return 'distinct city as node1, "alias" as label,
        kgtk_stringify(printf("%s, USA", kgtk_lqstring_text(city_label))) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`,
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort 
        -o "$TEMP"/derived.alias.city.USA.tsv.gz""")

In [40]:
kgtk("""--debug query --gc $STORE -i p31 -i p279star -i item -i label
        --match ' 
        p31: (city)-[]->(city_class), 
        p279star: (city_class)-[]->(:Q515), 
        item: (city)-[:P17]->(:Q30), 
        label: (city)-[]->(city_label)' 
        --return 'distinct city as node1, "alias" as label,
        kgtk_stringify(printf("%s, United States", kgtk_lqstring_text(city_label))) as `node2;kgtk:text`,
        "en" as `node2;kgtk:language`, 
        "language_qualified_string" as `node2;kgtk:data_type`'
        / implode -i - --mode NONE --types language_qualified_string --without language_suffix --remove-prefixed-columns True
        / add-id --id-style wikidata
        / sort
    -o "$TEMP"/derived.alias.city.united_states.tsv.gz""")

Combine the files for the US into one file, as it might be useful by itself

In [41]:
kgtk("""cat
        -i "$TEMP"/derived.alias.city.USA.tsv.gz
        -i "$TEMP"/derived.alias.city.united_states.tsv.gz
        / sort
        -o "$OUT"/derived.alias.city.us.tsv.gz""")

### `derived.alias.settlement.tsv` 
Combined file of all aliases for cities

In [42]:
kgtk("""cat
        -i "$OUT"/derived.alias.settlement.admin1.iso.tsv.gz 
        -i "$OUT"/derived.alias.settlement.admin1.short.tsv.gz 
        -i "$OUT"/derived.alias.settlement.admin1.full.tsv.gz 
        -i "$OUT"/derived.alias.city.country.tsv.gz 
        -i "$OUT"/derived.alias.city.us.tsv.gz 
        / compact -i - --deduplicate --build-id --id-style wikidata
        -o "$OUT"/derived.alias.settlement.tsv.gz""")

In [43]:
!$kgtk validate -i "$OUT"/derived.alias.settlement.tsv.gz


Data lines read: 1473081
Data lines passed: 1473081
